Per entrar a una sessió de Spark i iniciar un builder en DeltaLake

In [ ]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *

#!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"

builder = pyspark.sql.SparkSession.builder.appName("Shops_Deltalake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Comandes com si estessis a una sessió de Spark

Llegir els arxius i guardar-los en format DeltaLake, que ens garantitza que es compleixen les restriccions ACID, i dona avanatatges 




Carregar i llegir els 3 arxius de tenim

In [ ]:
#Arxiu Parquet 
income = spark.read.parquet("./datalake/income_data/2024-04-17_IRSIncomeByZipCode_NoStateTotalsNoSmallZips.parquet")
income.show()


In [4]:
from pyspark.sql.functions import col

# Lista de todos los caracteres inválidos que quieres reemplazar o eliminar
invalid_chars = [' ', ';', '{', '}', '(', ')', '\n', '\t', '=']

# Función para limpiar los nombres de las columnas reemplazando los caracteres no válidos
def clean_column_name(column_name):
    for invalid_char in invalid_chars:
        column_name = column_name.replace(invalid_char, "_")  # Reemplaza por subrayado o cualquier otro caracter válido que prefieras
    return column_name

# Aplicar la función de limpieza a cada columna
cleaned_income = income.select([col(c).alias(clean_column_name(c)) for c in income.columns])

# Ahora guarda el DataFrame limpio en formato Delta
# income.write.mode("overwrite").format("delta").save("./deltalake/income_data/")

In [ ]:
# finalitzar sessió de Spark
spark.stop()